#Step1:Define the functions we need.

#### allocate_teams

This is the main function and it assign the students in each tutorial group into 10 lists trying to meet the requirements to the maximum extent

1. Initial setup

`all_students`: a copy of `student_list`

`remaining_students`: a copy of `student_list`. To track students that haven't been assigned to a group.
`group_list`: initialize a nested list with 10 empty groups inside for future group assignment.


2. Assign the first 4 students of each group.

* the 1st and 3rd student:

 pick students with the highest cgpa, set index to -1,  pick from the end of the cgpa list.

* the 2nd and 4th student:
pick students with lowest cgpa, set index to 0, pick from the start of the cgpa list.

3. Add student to each group

Loop through the lists for every group in `group_list`, add current student to each group list.

4. Check school and gender of current list

If current list does not exceed gender and school threshold, add next student to the group.

Otherwise, try to swap student with another group. However, if swapper cannot find any valid target to swap with, then proceed to assign the next student.

5. Calculate the total cgpa of each current groups and store them with the group_num into group_cgpa.

6. Sort groups by their `total_gpa` in ascending orderand store the
groups in the listsort_output.

7. Assign the student based on gpa e.g. lowest gpa group picks highest gpa student

8. Check if the sutdent meets the gender or school requirement, if meets, assign student to the group whose index is group_num. If it dosen't meet, swap it using `swapper`.If `swapper` cannot find any valid target,just assign student. Finally return `group_list`.

Now we get a list `group_list`, including 10 lists, each list include 5 student.

In [44]:
def allocate_teams(student_list):
    all_students = student_list[:]
    remaining_students = student_list[:]
    group_list = [[] for i in range(10)]
    #assign 4 students to each group
    for i in range(4):
        #1st and 3rd pick take highest cgpa
        if i % 2 == 0:
            index = -1
        #2nd and 4th pick take lowest cgpa
        else:
            index = 0

        for group_num in range(len(group_list)):
            next_student = remaining_students[index]
            group = group_list[group_num]
            if not exceed_gender_or_school(group, next_student):
                assign_group(next_student, group_num, group, remaining_students)
            else:
                if swapper(next_student, all_students, remaining_students, group_list, group_num, group):
                    pass
                else:
                    #if swapper cant find any valid target, just assign
                    assign_group(next_student, group_num, group, remaining_students)


    group_cgpa = []
    for group_num in range(len(group_list)):
        total_cgpa = 0
        for student in group_list[group_num]:
            total_cgpa += student[1]
        group_cgpa.append([group_num, cgpa])

    sorted_output = []
    for group in group_cgpa:
        sort_by_ascending_cgpa(group, group[1], sorted_output)
    #sorted_output[[0, 16.4], [1, 16.5], [7, 16.7], ...]

    #each group left 1 slot, assign based on gpa e.g. lowest gpa group picks highest gpa student
    for i in range(len(sorted_output)):
        group_num = sorted_output[i][0]
        group = group_list[group_num]
        student = remaining_students[-1]
        if not exceed_gender_or_school(group, student):
            assign_group(student, group_num, group, remaining_students)
        else:
            if swapper(student, all_students, remaining_students, group_list, group_num, group):
                pass
            else:
                #if swapper cant find any valid target, just assign
                assign_group(student, group_num, group, remaining_students)

    return group_list

#### assign_group

This function is to assign student to the group and remove it from the remaining_students

1.Change the group_num of the student into the current group_num and append the student to the group whose index is group_num.

2.Remove the student from the remaining_students.

In [45]:
def assign_group(student, group_num, group, remaining_students):
    student[6] = group_num
    student[5] = True
    group.append(student)
    if student in remaining_students:
      remaining_students.remove(student)

#### swap_validator

This function supports the function `swapper()` . It is used to check if the try_student in the group can be replaced by student. If can ,the function will return True.

1.Check if the current group meets the requirement, if meets, return false.

2.Firstly, check if the try_student is in a group. If is, copy the group of try_student and remove try_student. Then check if we replace try_student with student, whether it can meet the requirement. If meets, return True.

3.If try_student is not in a group, return True.

In [46]:
def swap_validator(student, try_student, group_list, group):
    if exceed_gender_or_school(group, try_student):
        return False
    if try_student[5]:
        #create copy of try student group and remove try student to see
        #if student will fit in try student group after swap
        try_student_group_copy = group_list[try_student[6]][:]
        try_student_group_copy.remove(try_student)
        return not exceed_gender_or_school(try_student_group_copy, student)
    else:
        return True

#### swapper

This function is to swap student with the nearest gpa to meet the gender and school requirement.

1.Find the index of the student in the all_students list, initialize the parameters.

2.Loop through every students and check if there are still any more students of higher or lower gpa to check.

3.If there is a person whose gpa is higher than student, name this person try_student.Check if the student can be replaced by try_student.If can ,the swap_validator function will return True and we will assign True to swap. Then assign try_student to swap_student and break the loop.

4.If we cannot find a person with a higher gpa, we will check if there is someone with lower gpa. If there is ,simliarly ,we will name him try_student,check if student can be replaced by him .If can ,assign True to swap and assign try_student to swap_student and break the loop.

5.If the person with the nearest gpa with student cannot meet the requirement, increase the index_up and index_down to find the person who meets the requirement and have a nearer gpa.

6.If we find someone ,swap the student and the swap_student from their own groups and return True. If we cannot find a person suitable, return False.

In [47]:
def swapper(student, all_students, remaining_students, group_list, group_num, group):
    index = find_index(student, all_students)
    swap = False
    search_up, search_down = True, True
    index_up, index_down = 1, 1
    #loop through every student to check if can swap
    while search_up == True or search_down == True:
        #check if there are still any more students of higher/lower gpa to check
        if index + index_up > len(all_students) - 1:
            search_up = False
        if index - index_down < 0:
            search_down = False
        if search_up == True:
            try_student = all_students[index + index_up]
            if swap_validator(student, try_student, group_list, group):
                swap = True
                swap_student = try_student
                break
        if search_down == True:
            try_student = all_students[index - index_down]
            if swap_validator(student, try_student, group_list, group):
                swap = True
                swap_student = try_student
                break
        index_up += 1
        index_down += 1
    if swap == True:
        perform_swap(student, swap_student, remaining_students, group_num, group_list, group)
        return True
    else:
        return False

#### perform_swap

This function is to swap student and swap_student from their own group.

1.If the swap_student have already been assigned to a group,find his group and remove him from the group.

2.Then assign student to the group of swap_student and assign swap_student to the group whose index is group_num(the group which the student was supposed to be assigned to )

3.If the swap_student haven't been assigned to a group, then just assign him to the gourp instead of student.

In [48]:
def perform_swap(student, swap_student, remaining_students, group_num, group_list, group):
    if swap_student[5] == True:
        swap_student_group_num = swap_student[6]
        swap_student_group = group_list[swap_student_group_num]
        swap_student_group.remove(swap_student)
        assign_group(student, swap_student_group_num, swap_student_group, remaining_students)
        assign_group(swap_student, group_num, group, remaining_students)
    else:
        assign_group(swap_student, group_num, group, remaining_students)

#### find_index

This function supports the `swapper()` function, it finds the position of the student in the student_list and returns the index.

In [49]:
def find_index(value, list):
    for index in range(len(list)):
        if list[index] == value:
            return index

#### increase_count

This function supports the `exceed_gender_or_school()` function, it checks if the argument `key`(i.e. The school/gender of the student) is in the argument `dict`(The `schools`, `genders` dictionaries).

If the school of the student:
1.   already exists in the dictionary `schools`: add 1 to the value of the key(e.g."CCDS").
2.   is not in the dictionary `schools`: append a item with the school name as key and set the value to 1.

It works the same for gender.

In [50]:
def increase_count(dict, key):
    if key not in dict:
        dict[key] = 1
    else:
        dict[key] += 1

#### exceed_gender_or_school


check if the current group allocation have exceed gender or school. If there are more than 2 people from the same school, or more than 3 people of the same gender, return True.

1. define function with group and new_student as input. Create two empty dictionaries for school and gender to keep track of the count.

2. loop through all existing students in current group, examine the school and gender of the student. If the school/gender is already in the dictionaries, add 1 to the value of the key.

3. examine the school and gender  attribute of the new student, add 1 to the value of key in the dicitonary if it already exist in the current group.


4. check the final dictionary for school and gender, return True if there are more than 2 students from the same school or more than 3 students are the same gender.

In [51]:
def exceed_gender_or_school(group, new_student):
    schools = {}
    genders = {}
    #tally current genders and schools for each group
    for existing_student in group:
        #student[id, cgpa, gender, name, school, assigned, group number]
        increase_count(schools, existing_student[4])
        increase_count(genders, existing_student[2])
    #add gender and school of new student
    increase_count(schools, new_student[4])
    increase_count(genders, new_student[2])
    for school_count in schools.values():
        if school_count > 2:
            return True
    for gender_count in genders.values():
        if gender_count > 3:
            return True
    return False

#### sort_by_ascending_cgpa

This function supports the main function `allocate_teams()`. It sorts all groups in the list `group_cgpa` by their `total_cgpa` in ascending order and store the groups in the list `sorted_output`.

1. define the function.

Mapping to main function `allocate_teams()`:


*   `to_sort` -> `group` (e.g. [0, 16.4])
*   `cgpa` -> `group[1]` (e.g. 16.4)
*   `output` -> `sorted_output`(e.g. [[0, 16.4]])

If output is empty, add current group to the output list.

loop through existing output list, find wherer to insert current entry based on cgpa.

Let current entry(cgpa) move right in the list until it is less than or equal to another group's cgpa.

In [52]:
def sort_by_ascending_cgpa(to_sort, cgpa, output):
    if output == []:
        output.append(to_sort)
        return
#loop through each existing entry to find where to insert current entry based on cgpa
    index = 0
    for entry in output:
        entry_cgpa = entry[1]
        if cgpa > entry_cgpa:
            #move right in list
            index += 1
            #at end of list
            if index == len(output):
                output.append(to_sort)
                return
            continue
        elif cgpa <= entry_cgpa:
            output.insert(index, to_sort)
            return

In [53]:
import matplotlib.pyplot as plt
def allocate_teams(student_list):
    all_students = student_list[:]
    remaining_students = student_list[:]
    group_list = [[] for i in range(10)]
    #assign 4 students to each group
    for i in range(4):
        #1st and 3rd pick take highest cgpa
        if i % 2 == 0:
            index = -1
        #2nd and 4th pick take lowest cgpa
        else:
            index = 0
    
        for group_num in range(len(group_list)):
            next_student = remaining_students[index]
            group = group_list[group_num]
            if not exceed_gender_or_school(group, next_student):
                assign_group(next_student, group_num, group, remaining_students)
            else:
                if swapper(next_student, all_students, remaining_students, group_list, group_num, group):
                    pass
                else:
                    #if swapper cant find any valid target, just assign
                    assign_group(next_student, group_num, group, remaining_students)


    group_cgpa = []
    for group_num in range(len(group_list)):
        total_cgpa = 0
        for student in group_list[group_num]:
            total_cgpa += student[1]
        group_cgpa.append([group_num, cgpa])

    sorted_output = []
    for group in group_cgpa:
        sort_by_ascending_cgpa(group, group[1], sorted_output)
    #sorted_output[[0, 16.4], [1, 16.5], [7, 16.7], ...]

    #each group left 1 slot, assign based on gpa e.g. lowest gpa group picks highest gpa student
    for i in range(len(sorted_output)):
        group_num = sorted_output[i][0]
        group = group_list[group_num]
        student = remaining_students[-1]
        if not exceed_gender_or_school(group, student):
            assign_group(student, group_num, group, remaining_students)
        else:
            if swapper(student, all_students, remaining_students, group_list, group_num, group):
                pass
            else:
                #if swapper cant find any valid target, just assign
                assign_group(student, group_num, group, remaining_students)            

    return group_list

def assign_group(student, group_num, group, remaining_students):
    student[6] = group_num
    student[5] = True
    group.append(student)
    if student in remaining_students:
        remaining_students.remove(student)

def swap_validator(student, try_student, group_list, group):
    if exceed_gender_or_school(group, try_student):
        return False
    
    if try_student[5]:
        #create copy of try student group and remove try student to see 
        #if student will fit in try student group after swap
        try_student_group_copy = group_list[try_student[6]][:]
        try_student_group_copy.remove(try_student)
        return not exceed_gender_or_school(try_student_group_copy, student)
    else:
        return True

def swapper(student, all_students, remaining_students, group_list, group_num, group):
    index = find_index(student, all_students)
    swap = False
    search_up, search_down = True, True
    index_up, index_down = 1, 1
    #loop through every student to check if can swap
    while search_up == True or search_down == True:
        #check if there are still any more students of higher/lower gpa to check
        if index + index_up > len(all_students) - 1:
            search_up = False
        if index - index_down < 0:
            search_down = False
        
        if search_up == True:
            try_student = all_students[index + index_up]
            if swap_validator(student, try_student, group_list, group):
                swap = True
                swap_student = try_student
                break

        if search_down == True:
            try_student = all_students[index - index_down]
            if swap_validator(student, try_student, group_list, group):
                swap = True
                swap_student = try_student
                break

        index_up += 1
        index_down += 1

    if swap == True:
        perform_swap(student, swap_student, remaining_students, group_num, group_list, group)
        return True
    else:
        return False

def perform_swap(student, swap_student, remaining_students, group_num, group_list, group):
    if swap_student[5] == True:
        swap_student_group_num = swap_student[6]
        swap_student_group = group_list[swap_student_group_num]
        swap_student_group.remove(swap_student)
        assign_group(student, swap_student_group_num, swap_student_group, remaining_students)
        assign_group(swap_student, group_num, group, remaining_students)
    else:
        assign_group(swap_student, group_num, group, remaining_students)

def find_index(value, list):
    for index in range(len(list)):
        if list[index] == value:
            return index

def increase_count(dict, key):
    if key not in dict:
        dict[key] = 1
    else:
        dict[key] += 1

def exceed_gender_or_school(group, new_student):
    schools = {}
    genders = {}
    #tally current genders and schools for each group
    for existing_student in group:
        #student[id, cgpa, gender, name, school, assigned, group number]
        increase_count(schools, existing_student[4])
        increase_count(genders, existing_student[2])
    #add gender and school of new student
    increase_count(schools, new_student[4])
    increase_count(genders, new_student[2])
    for school_count in schools.values():
        if school_count > 2:
            return True
    for gender_count in genders.values():
        if gender_count > 3:
            return True
    return False

def sort_by_ascending_cgpa(to_sort, cgpa, output):
    if output == []:
        output.append(to_sort)
        return
        
    #loop through each existing entry to find where to insert current entry based on cgpa
    index = 0
    for entry in output: 
        entry_cgpa = entry[1]
        if cgpa > entry_cgpa:
            #move right in list
            index += 1
            #at end of list
            if index == len(output):
                output.append(to_sort)
                return
            continue
        elif cgpa <= entry_cgpa:
            output.insert(index, to_sort)
            return
        
#BELOW IS TO RUN AND ANALYSE

from read_to_dict import read_to_dict
import pprint

student_data_dict = read_to_dict("records.csv")

successful = 0
unsuccessful = 0
half = 0
stdevs = []
final_data = []
for tgnum in student_data_dict:
    output = []
    tutorial_group = student_data_dict[tgnum]
    for id, data in tutorial_group.items():
        student_data = [id, data['CGPA'], data['Gender'], data['Name'], data['School'], data['Assigned'], data['Group Number']]
        cgpa = student_data[1]
        sort_by_ascending_cgpa(student_data, cgpa, output)
    groupz = (allocate_teams(output))
    for group in groupz:
        for student in group:
            final_student_data = [tgnum, student[0],student[4],student[3],student[2],str(student[1]),str(student[6])] #“Tutorial Group”, “Student ID”, “School”, “Name”, “Gender”, “CGPA”, "Team Assigned"
            final_data.append(final_student_data)
    cgpas = []
    
    for group in groupz:
        schoolpass = True
        genderpass = True
        cgpa = 0
        schools = {}
        genders = {}
        
        #tally current genders and schools for each group
        for student in group:
            cgpa += student[1]
            #student[id, cgpa, gender, name, school, assigned, group number]
            increase_count(schools, student[4])
            increase_count(genders, student[2])
        #add gender and school of new student
        for school_count in schools.values():
            if school_count > 2:
                schoolpass = False
        for gender_count in genders.values():
            if gender_count > 3:
                genderpass = False  
        if not schoolpass and not genderpass:
            unsuccessful += 1
        elif not schoolpass:
            half += 1
        elif not genderpass:
            half += 1
        else:
            successful += 1

        cgpas.append(cgpa)


    # Step 1: Compute the mean
    mean = sum(cgpas) / 10

    # Step 2: Calculate squared differences
    squared_diffs = [(x - mean) ** 2 for x in cgpas]

    # Step 3: Calculate variance
    variance = sum(squared_diffs) / len(data)

    # Step 4: Calculate standard deviation
    stdev = variance ** 0.5
    stdevs.append(stdev)
        
with open('new_records.csv','w') as f:
    f.write('Tutorial Group,Student ID,School,Name,Gender,CGPA,Team Assigned\n')
    for i in final_data:
        f.writelines(','.join(i))
        f.write('\n')
            
print(f"{successful} are successful, {unsuccessful} are unsuccessful and {half} are half successful")
print(stdevs)
labels = ['Successful', 'Half Successful', 'Not Successful']
sizes = [successful,half,unsuccessful ]

# Calculate percentages, ensuring they sum to exactly 100%
total = sum(sizes)
percentages = [size / total * 100 for size in sizes]
percentages[-1] = 100 - sum(percentages[:-1])  # Adjust last percentage to ensure total is 100%


# Create the formatted labels with percentages for the legend
legend_labels = [f'{label}: {percentage:.1f}%' for label, percentage in zip(labels, percentages)]

# Define explode, colors, and start angle
explode = [0, 0.5, 1]  # "Explode" the first slice (Category A)
colors = ['#99ff99', '#E1E100', '#ff9999']
startangle = 140  # Rotate the chart so the first slice starts at 90 degrees

# Plot the pie chart with explode, colors, and startangle
fig, ax = plt.subplots()
wedges, texts= ax.pie(sizes, labels=labels,explode=explode,colors=colors, startangle=startangle)


# Add the legend with percentage labels
ax.legend(wedges, legend_labels, title="Distribution of Success Cases", loc="best")
plt.axis('equal')
plt.show()

averageSD= sum(stdevs)/len(stdevs)
print(averageSD)
averagegpa= sum(cgpas)/len(cgpas)

ModuleNotFoundError: No module named 'read_to_dict'